In [118]:
import pandas as pd
import time

# Daten zusammenführen
Alle unsere Daten:

In [119]:
bna = pd.read_csv("cleaned_data/StromVersorgerListe_Bundesnetzagentur-cleaned-2019-08-07.csv")
vv = pd.read_csv("cleaned_data/verivox-alle-anbieter-cleaned-2019-08-19.csv")
c24 = pd.read_csv("raw_data/check24-alle-anbieter-scaped-2019-08-07.csv")
sa1 = pd.read_csv("cleaned_data/Stromauskunftde-oekostrom-cleaned-2019-08-08.csv")
sa2 = pd.read_csv("cleaned_data/stromauskunftde-reine-oekostrom-cleaned-2019-08-06.csv")
ett = pd.read_csv("cleaned_data/ecotopten_oekostrom_cleaned-2019-08-19.csv")
oet = pd.read_csv("cleaned_data/oekotest-strom-cleaned-2019-08-19.csv")
ut = pd.read_csv("cleaned_data/utopia-bestenliste-cleaned-2019-08-19.csv")
op = pd.read_csv("cleaned_data/okpower-cleaned-2019-07-30.csv")
gs = pd.read_csv("cleaned_data/gruenstrom-cleaned-2019-07-30.csv")

## Stromanbieter

Wir haben ein paar Quellen, die nur Stromanbieter im Allgemeinen auflisten.
Leider machen es die unterschiedlichen Arten der Benennung sehr schwer die Listen einigermaßen brauchbar zusammenzuführen.

Gerade der von Check24 gescrapte Datensatz ist fast unbrauchbar. Verivox sieht da vielversprechender aus.


## Ökostromanbieter

Widmen wir uns den Ökostrom(tarif|produkt)anbieterlisten. Mal sehen!

### Stromauskunft + Verivox

Zuerst die großen Listen Stromauskunft `sa1` und Verivox `vv` mit Firmenadressen!

In [208]:
# Zuerst Spalten umbenennen
sa1.rename(columns={"Stromanbieter": "Firmenname",
                   "Adresse 1": "Firmenanschrift",
                   "Stadt": "Firmenort"}, inplace=True)

sa1['Stromauskunft'] = True

# Dann mergen
sa_vv_list = pd.merge(sa1,
               vv[['Firmenname', 'Verivox']],
               on='Firmenname',
               how='outer')

print("Verivox dimensions {}".format(vv.shape))
print("Stromauskunft dimensions {}".format(sa1[["Firmenname"]].shape))
print("Ergebnis {}".format(sa_vv_list.shape))

print("Write to csv")
sa_vv_list.to_csv("merged_data/sa_vv_list-{}.csv".format(time.strftime("%Y%m%d-%H%M%S")))

Verivox dimensions (1147, 2)
Stromauskunft dimensions (715, 1)
Ergebnis (1157, 6)
Write to csv


Sehr viele Überschneidungen! Wir belassen das so, ohne nachzuarbeiten.


### Kleine Listen zusammenführen

Die Liste der "echten" Ökostromanbieter von Stromauskunft (`sa2`), EcoTopTen(`ett`), Ökotest (`oet`) und Utopia (`ut`) sind nicht sehr lang. Evtl können wir hier einfach mergen und manuell weitermachen.

In [159]:
# Spalten umbenennen
sa2['Stromauskunft echter Oekostrom'] = True
sa2.rename(columns={"Anbieter": "Firmenname"}, inplace=True)
print("Stromauskunft dimensions {}".format(sa2.shape))

ett['EcoTopTen'] = True
print("EcoTopTen dimensions {}".format(ett[['Firmenname', 'EcoTopTen']].shape))

oet['Ökotest'] = True
oet.rename(columns={"Anbieter": "Firmenname"}, inplace=True)
print("Ökotest dimensions {}".format(oet.shape))

ut['Utopia'] = True
ut.rename(columns={"Name": "Firmenname"}, inplace=True)
print("Utopia dimensions {}".format(ut[['Firmenname', 'Utopia']].shape))  

Stromauskunft dimensions (18, 3)
EcoTopTen dimensions (30, 2)
Ökotest dimensions (34, 3)
Utopia dimensions (16, 2)


In [165]:
# Mergen!
small_list = pd.merge(oet[['Firmenname', 'Ökotest']],
               ett[['Firmenname', 'EcoTopTen']],
               on='Firmenname',
               how='outer')

print("List dimensions oet + ett {}".format(small_list.shape))

small_list = pd.merge(small_list,
                     ut[['Firmenname', 'Utopia']],
                     on='Firmenname',
                     how='outer')

print("List dimensions + ut {}".format(small_list.shape))

small_list = pd.merge(small_list,
                     sa2[['Firmenname', 'Stromauskunft echter Oekostrom']],
                     on='Firmenname',
                     how='outer')

print("List dimensions + sa2 {}".format(small_list.shape))

print("Write to csv")
small_list.to_csv("merged_data/small_list-{}.csv".format(time.strftime("%Y%m%d-%H%M%S")))

List dimensions oet + ett (62, 3)
List dimensions + ut (74, 4)
List dimensions + sa2 (85, 5)
Write to csv


Leider haben wir auch hier das Problem der unterschiedlichen Firmen- und Markennamen. In Open Refine habe ich die Liste manuell gesäubert und zusammengefasst. Es stechen nur zwei Anbieter hervor, die nur in jeweils einer Liste auftauchen und nicht mit anderen in Verbindung stehen.

In [213]:
small_list_clean = pd.read_csv("merged_data/small_list-oet-ett-ut-st2-20190821-cleaned.csv")
print(small_list_clean.shape)

(36, 6)


### Labellisten

In [187]:
op['OK Power'] = True
op.rename(columns={"Anbieter": "Firmenname"}, inplace=True)
print("OK power dimensions {}".format(op.shape))


gs['Grüner Strom'] = True
gs.rename(columns={"Anbieter": "Firmenname"}, inplace=True)
print("Grüner Strom dimensions {}".format(gs.shape))

label_list = pd.merge(op[['Firmenname', 'Adresse', 'Telefon', 'Website', 'OK Power']],
                     gs[['Firmenname', 'Verfügbarkeit', 'Telefon', 'Website', 'Grüner Strom']],
                     on='Firmenname',
                     how='outer')

print("Label Liste dimensions {}".format(label_list.shape))

print("Write to csv")
label_list.to_csv("merged_data/label_list-{}.csv".format(time.strftime("%Y%m%d-%H%M%S")))


OK power dimensions (75, 6)
Grüner Strom dimensions (64, 6)
Label Liste dimensions (137, 9)


Auch hier müssen wir manuell ran und die Telefonnummers und Websites in Open Refine umtragen.


In [214]:
label_list_clean = pd.read_csv("merged_data/label_list-20190821-cleaned.csv")
print(label_list_clean.shape)

(134, 10)


### Labellisten und kleine Liste

Wir arbeiten weiter mit den gesäuberten Listen der letzten beiden Schritte.

In [193]:
#small_list_clean = pd.read_csv("merged_data/small_list-oet-ett-ut-st2-20190821-cleaned.csv")
#label_list_clean = pd.read_csv("merged_data/label_list-20190821-cleaned.csv")

oeko_list = pd.merge(small_list_clean,
                    label_list_clean,
                    on="Firmenname",
                    how="outer")

print("Small Lis dimensions {}".format(small_list_clean.shape))
print("Label Liste dimensions {}".format(label_list_clean.shape))
print("Öko Liste dimensions {}".format(oeko_list.shape))

print("Write to csv")
oeko_list.to_csv("merged_data/oeko_list-{}.csv".format(time.strftime("%Y%m%d-%H%M%S")))


Small Lis dimensions (36, 6)
Label Liste dimensions (134, 10)
Öko Liste dimensions (161, 15)
Write to csv


Hier hatte ich mehr Überschneidungen erwartet, schließlich sollten die Anbieter auf der kleinen Liste zum Großteil in der Labelliste vertreten sein. Zeit in Open Refine nachzuarbeiten.

In [215]:
oeko_list_clean = pd.read_csv("merged_data/oeko_list-20190821-cleaned.csv")
print(oeko_list_clean.shape)

(144, 13)


### Große Anbieterliste
Jetzt legen wir diese auf die große Anbieterliste aus Vervox und Stromauskunft.de

In [216]:
oeko_list_clean = pd.read_csv("merged_data/oeko_list-20190821-cleaned.csv")

oeko_list_large = pd.merge(oeko_list_clean,
                          sa_vv_list,
                          on="Firmenname",
                          how="outer")

print("SA VV Liste dimensions {}".format(sa_vv_list.shape))
print("kleine Öko Liste dimensions {}".format(oeko_list_clean.shape))
print("Große Öko Liste dimensions {}".format(oeko_list_large.shape))

print("Write to csv")
oeko_list_large.to_csv("merged_data/oeko_list_large-{}.csv".format(time.strftime("%Y%m%d-%H%M%S")))

SA VV Liste dimensions (1157, 6)
kleine Öko Liste dimensions (144, 13)
Große Öko Liste dimensions (1226, 18)
Write to csv


Wieder trefen ein paar Dopplungen auf, auch die Spalten machen jetzt ärger. In Open Refine wird beides so gut es geht bereinigt.

In [211]:
list_final = pd.read_csv("merged_data/oeko_list_large-20190826-cleaned.csv")
print(list_final.shape)
list_final.head()

(1202, 15)


,Firmenname,Grüner Strom,OK Power,Stromauskunft,Verivox,Ökotest,EcoTopTen,Utopia,Stromauskunft echter Oekostrom,Telefon,Website,Verfügbarkeit,Firmenanschrift,PLZ,Firmenort
0,123energie - eine Marke der Pfalzwerke AG,NaN,True,True,True,True,True,NaN,NaN,0621 570573123,www.123energie.de,NaN,Kurfürstenstraße 29,67061,Ludwigshafen
1,17er Oberlandenergie GmbH,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Viehmarktplatz 1,82418,Murnau am Staffelsee
2,24/7 Energie und Kommunikation GmbH,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Meeboldstraße 1,89522,Heidenheim
3,4hundred GmbH,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abens-Donau Energie GmbH,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
